In [1]:
# параметры подключения к БД (извините, секрет)
PASSWORD = ''
HOST = ''
USER = ''
DB = ''

In [500]:
mysql_connect = mysql.connector.connect(user=USER, password=PASSWORD,
                                            host=HOST,
                                            port=3308,
                                            database=DB, auth_plugin='mysql_native_password')
cursor = mysql_connect.cursor()

# Часть 1. Получаем данные

└[∵┌]└[ ∵ ]┘[┐∵]┘

# Данные
https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

In [3]:
import pandas as pd
from ast import literal_eval
cred = pd.read_csv('tmdb_5000_credits.csv')
cred.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [4]:
meta = pd.read_csv('tmdb_5000_movies.csv')
meta.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


# Данные для таблиц characters, actors, movies_characters

In [17]:
from tqdm import tqdm

In [18]:
s = 0 # счетчик
movies_chars = [] # id фильмов 
name_chars = [] # имя персонажа
actor_chars = [] # имя актера
gender_chars = [] # пол

for item in tqdm(cred['cast']):
    pairs = item.split(',')
    if str(pairs).count('name') != str(pairs).count('gender'):
        pass
    else:
        for p in pairs:
            if "character" in p:
                name_chars.append(p.split(": ")[1].replace('"', ''))
                movies_chars.append(cred["movie_id"][s])
            if "name" in p:
                actor_chars.append(p.split(": ")[1].replace('"', '')) 
            if "gender" in p:
                gender_chars.append(p.split(": ")[1].replace('"', '')) 
        s += 1

100%|█████████████████████████████████████| 4803/4803 [00:02<00:00, 1612.81it/s]


In [23]:
s = 459488
id_chars = []
for i in movies_chars:
    s+=1
    id_chars.append(s)

## Таблица Персонажей

In [252]:
df_chars = pd.DataFrame({
    'char_id': list(dict(zip(name_chars, id_chars)).values()),
    'movie_id': list(dict(zip(name_chars, movies_chars)).values()),
    'char_name': list(dict(zip(name_chars, id_chars)).keys()),
    'gender': list(dict(zip(name_chars, gender_chars)).values()),
    'actor_name': list(dict(zip(name_chars, actor_chars)).values())
    
            })

df_chars.head(3)

,char_id,movie_id,char_name,gender,actor_name
0,459489,19995,Jake Sully,2,Sam Worthington
1,459490,19995,Neytiri,1,Zoe Saldana
2,459491,19995,Dr. Grace Augustine,1,Sigourney Weaver


## Таблица актеров

Хочется получить что-то помимо имен актеров - воспользуемся википедией :)

Обкачаем даты рождения актеров, используя модули wikipedia и datefinder

In [253]:
id_actors = []
s = max(id_chars)
for i in set(actor_chars):
    s+=1
    id_actors.append(s)
len(id_actors)

53737

In [254]:
len(list(dict(zip(id_actors, actor_chars))))

53737

**Вариант 1 - обкачать википедию с датами рождения актеров, но это долго (около 40 часов)**

In [255]:
import wikipedia
import datefinder
birth_date = []
for actor in tqdm(list(dict(zip(id_actors, actor_chars)).values())[0:6]): # протестим на 6 элементах списка
    try:
        input_string = wikipedia.page(f"{actor}").summary
        matches = list(datefinder.find_dates(input_string))
        if len(matches) > 0:
            birth_date.append(matches[0])
        else:
            birth_date.append('')
    except Exception:
        birth_date.append('')

100%|█████████████████████████████████████████████| 6/6 [00:04<00:00,  1.20it/s]


In [256]:
birth_date

[datetime.datetime(1976, 8, 2, 0, 0),
 datetime.datetime(1978, 6, 19, 0, 0),
 datetime.datetime(1949, 10, 8, 0, 0),
 '',
 datetime.datetime(1978, 7, 12, 0, 0),
 datetime.datetime(1974, 12, 17, 0, 0)]

**Вариант 2 - mock данные**

In [257]:
from faker import Faker
fake = Faker()
birth_date = []
for actor in tqdm(list(dict(zip(id_actors, actor_chars)).values())):
    birth_date.append(fake.date_time_between(start_date='-70y', end_date='-10y').date())

100%|██████████████████████████████████| 53737/53737 [00:02<00:00, 23781.95it/s]


In [258]:
birth_date[0]

datetime.date(1998, 4, 22)

In [259]:
df_actors = pd.DataFrame({
    'actor_id': list(dict(zip(id_actors, actor_chars)).keys()),
    'char_id': list(dict(zip(id_actors, id_chars)).values()),
    'actor_name': list(dict(zip(id_actors, actor_chars)).values()),
    'birth_date': birth_date
            })

df_actors.head(3)

,actor_id,char_id,actor_name,birth_date
0,564842,459489,Sam Worthington,1998-04-22
1,564843,459490,Zoe Saldana,2007-05-12
2,564844,459491,Sigourney Weaver,1992-12-15


## Джойним датафреймы

нужно в df_chars id актеров, а не имена

In [260]:
df_char = df_chars.merge(df_actors, on='actor_name', how='left')
df_char.head(3)

,char_id_x,movie_id,char_name,gender,actor_name,actor_id,char_id_y,birth_date
0,459489,19995,Jake Sully,2,Sam Worthington,564842.0,459489.0,1998-04-22
1,459489,19995,Jake Sully,2,Sam Worthington,567225.0,461872.0,1979-03-19
2,459489,19995,Jake Sully,2,Sam Worthington,570991.0,465638.0,1956-03-24


In [262]:
df_char = df_char[['char_id_x', 'movie_id', 'char_name', 'gender']].copy()

In [268]:
df_chars =  pd.DataFrame({
    'char_id': list(dict(zip(OrderedSet(df_char['char_name']), df_char['char_id_x'])).values()),
    'movie_id': list(dict(zip(OrderedSet(df_char['char_name']), df_char['movie_id'])).values()),
    'name': list(dict(zip(OrderedSet(df_char['char_name']), df_char['gender'])).keys()),
    'gender': list(dict(zip(OrderedSet(df_char['char_name']), df_char['gender'])).values())})
df_chars

,char_id,movie_id,name,gender
0,459489,19995,Jake Sully,2
1,459489,19995,Neytiri,2
2,459489,19995,Dr. Grace Augustine,2
3,459489,19995,Col. Quaritch,2
4,459489,19995,Trudy Chacon,2
...,...,...,...,...
70820,478917,395,Rita Haywith,2
70821,478917,395,Norman Dorman,2
70822,478917,395,Charlie Riggs,2
70823,478917,395,Andrea Shmeckle,2


## Таблица фильмов

In [323]:
df_movies = meta[['id', 'release_date', 'title', 'budget', 
                  'revenue', 'runtime', 'vote_average', 'vote_count']].copy()
df_movies.head(3)

,id,release_date,title,budget,revenue,runtime,vote_average,vote_count
0,19995,2009-12-10,Avatar,237000000,2787965087,162.0,7.2,11800
1,285,2007-05-19,Pirates of the Caribbean: At World's End,300000000,961000000,169.0,6.9,4500
2,206647,2015-10-26,Spectre,245000000,880674609,148.0,6.3,4466


In [324]:
df_movies['movie_id'] = df_movies['id']

## Таблица студий 

In [81]:
s = 0
comp = []
ids = []
for item in tqdm(meta['production_companies']):
    pairs = item.split(',')
    for p in pairs:
        if "name" in p:
            comp.append(p.split(": ")[1].replace('"', '').replace('}', '').replace(']', ''))
            ids.append(meta['id'][s])
    s += 1

100%|████████████████████████████████████| 4803/4803 [00:00<00:00, 19350.16it/s]


In [89]:
s = max(ids)
c_id = []
for i in comp:
    s+=1
    c_id.append(s)

In [92]:
df_comp = pd.DataFrame({
    'comp_id': c_id,
    'name': comp,
    'movie_id': ids
})
df_comp.head(3)

,comp_id,name,movie_id
0,459489,Ingenious Film Partners,19995
1,459490,Twentieth Century Fox Film Corporation,19995
2,459491,Dune Entertainment,19995


In [158]:
country = []
for i in comp:
    country.append(fake.country())

In [159]:
df_comp = pd.DataFrame({
    'comp_id': c_id,
    'name': comp,
    'movie_id': ids,
    'country': country
    
})
df_comp.head(3)

,comp_id,name,movie_id,country
0,459489,Ingenious Film Partners,19995,Azerbaijan
1,459490,Twentieth Century Fox Film Corporation,19995,United Kingdom
2,459491,Dune Entertainment,19995,Argentina


In [160]:
from ordered_set import OrderedSet

In [329]:
df_movies_final = df_movies_final[['movie_id', 'comp_id', 'release_date', 'title', 'budget', 'revenue', 
                                   'runtime', 'vote_average', 'vote_count']]
df_movies_final.head(1)

,movie_id,comp_id,release_date,title,budget,revenue,runtime,vote_average,vote_count
0,19995,459489,2009-12-10,Avatar,237000000,2787965087,162.0,7.2,11800


In [163]:
df_comp_final =  pd.DataFrame({
    'comp_id': list(dict(zip(OrderedSet(comp), c_id)).values()),
    'name': list(dict(zip(OrderedSet(comp), c_id)).keys()),
    'country': list(dict(zip(OrderedSet(comp), country)).values())
    
})
df_comp_final.tail(3)

,comp_id,name,country
5009,464498,Front Street Pictures,Oman
5010,464499,rusty bear entertainment,New Zealand
5011,464500,lucky crow films,Iraq


In [110]:
del df_comp

(ﾉﾟ▽ﾟ)ﾉ

# Часть 2. Создание таблиц в базе данных

### CRUD

**Create**

In [218]:
command = ("""
                CREATE TABLE heheHAHA (
                hehe_id INT NOT NULL PRIMARY KEY,
                hehehe VARCHAR(255)
                ) 
                """)
cursor.execute(command)
mysql_connect.commit()

**Update**

In [219]:
command = " UPDATE heheHAHA SET hehe_id = 1, hehehe='HEHEHEHEH' "
cursor.execute(command)
mysql_connect.commit()

**Delete**

In [220]:
command = " DELETE FROM heheHAHA WHERE hehe_id = 1 "
cursor.execute(command)
mysql_connect.commit()

**Drop**

Ой! Нам ведь вообще не нужна эта таблица - дропнем ее

In [375]:
command = "DROP TABLE heheHAHA"
cursor.execute(command)
mysql_connect.commit()

Создаем реальные таблицы

In [319]:
command = ("""
                CREATE TABLE IF NOT EXISTS companie (
                comp_id INT NOT NULL PRIMARY KEY,
                name VARCHAR(128),
                country VARCHAR(56)
                ) 
                """)
cursor.execute(command)
mysql_connect.commit()

In [376]:
command = ("""
                CREATE TABLE IF NOT EXISTS movie (
                movie_id INT NOT NULL,
                comp_id INT NOT NULL,
                release_date DATE,
                title VARCHAR(128),
                budget BIGINT,
                revenue BIGINT,
                runtime INT,
                vote_average INT,
                vote_count INT,
                PRIMARY KEY (movie_id),
                FOREIGN KEY (comp_id) REFERENCES companie(comp_id)               
                ) 
                """)
cursor.execute(command)
mysql_connect.commit()

In [332]:
command = ("""
                CREATE TABLE IF NOT EXISTS characters_in_movies (
                char_id INT NOT NULL,
                movie_id INT NOT NULL,
                name VARCHAR(255),
                gender INT,
                hehehe CHAR(50),
                PRIMARY KEY (char_id),
                FOREIGN KEY (movie_id) REFERENCES movie(movie_id)
                ) 
                """)
cursor.execute(command)
mysql_connect.commit()

зачем колонка hehehe? Чтобы использовать alter

**Alter**

In [335]:
command = 'ALTER TABLE characters_in_movies DROP COLUMN hehehe'
cursor.execute(command)
mysql_connect.commit()

In [244]:
command = ("""
                CREATE TABLE IF NOT EXISTS actor (
                actor_id INT NOT NULL,
                char_id INT NOT NULL,
                name VARCHAR(255),
                birth_date DATE,
                PRIMARY KEY (actor_id),
                FOREIGN KEY (char_id) REFERENCES characters_in_movies(char_id)
                ) 
                """)
cursor.execute(command)
mysql_connect.commit()

#  Часть 3. Запись в БД

In [229]:
def split_dataframe(df, chunk_size=5000):
    """
    Запись максимум N строк за раз
    """
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i * chunk_size:(i+1) * chunk_size])
    return chunks

Таблица companie

In [346]:
df_chunks = split_dataframe(df_comp_final)

In [347]:
for df in tqdm(df_chunks):
    query = 'INSERT INTO companie (comp_id,name,country) VALUES (%s, %s, %s)'
    cursor.executemany(query, df.values.tolist())
    mysql_connect.commit()

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  5.66it/s]


Таблица movie

In [377]:
df_chunks = split_dataframe(df_movies_final)

In [378]:
for df in tqdm(df_chunks):
    query = 'INSERT INTO movie (movie_id,comp_id,release_date,title,budget,revenue,runtime,vote_average,vote_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'
    cursor.executemany(query, df.values.tolist())
    mysql_connect.commit()

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  1.97it/s]


Таблица actor

In [250]:
df_chunks = split_dataframe(df_actors)

In [251]:
for df in tqdm(df_chunks):
    query = 'INSERT INTO actor (actor_id, char_id, name, birth_date) VALUES (%s, %s, %s, %s)'
    cursor.executemany(query, df.values.tolist())
    mysql_connect.commit()

100%|███████████████████████████████████████████| 11/11 [00:03<00:00,  3.51it/s]


Таблица character

In [338]:
df_chunks = split_dataframe(df_chars.drop_duplicates(subset=['char_id']))

In [339]:
for df in tqdm(df_chunks):
    query = 'INSERT INTO characters_in_movies (char_id, movie_id, name, gender) VALUES (%s, %s, %s, %s)'
    cursor.executemany(query, df.values.tolist())
    mysql_connect.commit()

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.97it/s]


*:・ﾟ✧(∩｀-´)⊃━☆ﾟ.*･｡ﾟ☆ﾟ. * ･ ｡ﾟ,☆ﾟ. * ･ ｡ﾟ,☆ﾟ. * ･ ｡ﾟ,

# Часть 4. Запросы

**1. Сколько персонажей было в фильмах от Корейских компаний?**

In [569]:
q = ("""
    SELECT COUNT(DISTINCT(characters_in_movies.char_id)) 
    FROM characters_in_movies, 
         movie, 
         companie 
    WHERE characters_in_movies.movie_id = movie.movie_id 
      AND movie.comp_id = companie.comp_id 
      AND companie.country = 'Korea' 
      """)
cursor.execute(q)
data = cursor.fetchone()
print('Количество персонажей в фильмах от Корейских компаний:', data[0])

Количество персонажей в фильмах от Корейских компаний: 440


**2. Какие средние бюджеты компаний из Канады и США (и при этом средний бюджет больше 80000000) с сортировкой от наибольшего среднего бюджета к наименьшему?**

In [570]:
q = ("""
    SELECT avg_budget,
       comp
    FROM
      (SELECT AVG(movie.budget) AS avg_budget,
          companie.name AS comp
       FROM movie,
        companie
       WHERE movie.comp_id=companie.comp_id
         AND companie.country IN ('Canada', 'USA')
       GROUP BY comp) t
    WHERE avg_budget > 80000000
    ORDER BY avg_budget DESC 
    """)
cursor.execute(q)
data = cursor.fetchall()
for i in data:
    print('Avg. budget: ', i[0], 'Companie name: ', i[1])
    print('---------------------------------------------------------------------')

Avg. budget:  210000000.0000 Companie name:  Centropolis Entertainment
---------------------------------------------------------------------
Avg. budget:  120000000.0000 Companie name:  Envision Entertainment
---------------------------------------------------------------------
Avg. budget:  100000000.0000 Companie name:  Really Useful Films
---------------------------------------------------------------------
Avg. budget:  100000000.0000 Companie name:  Hyde Park Films
---------------------------------------------------------------------
Avg. budget:  90000000.0000 Companie name:  Cube Vision
---------------------------------------------------------------------
Avg. budget:  90000000.0000 Companie name:  Count of Monte Cristo Ltd.
---------------------------------------------------------------------


**3. У какого фильма самый большой бюджет и какая у него средняя оценка зрителей (ну и число проголосовавших) ?**

In [571]:
q = ("""
    SELECT title,
       budget,
       vote_average,
       vote_count
    FROM movie
    WHERE budget =
        (SELECT max(budget)
         FROM movie)
         """)
cursor.execute(q)
data = cursor.fetchall()
print(data[0][0], '| Budget: ', data[0][1], '| Vote Avg.: ', data[0][2], '| Vote Count: ', data[0][3])

Pirates of the Caribbean: On Stranger Tides | Budget:  380000000 | Vote Avg.:  6 | Vote Count:  4948


**4. А по самой большой выручке?**

(запрос простой, но ведь интересно....)

In [489]:
q = ("""
    SELECT title,
       revenue,
       vote_average,
       vote_count
    FROM movie
    WHERE revenue =
        (SELECT max(revenue)
         FROM movie)
         """)
cursor.execute(q)
data = cursor.fetchall()
print(data[0][0], '| Revenue: ', data[0][1], '| Vote Avg.: ', data[0][2], '| Vote Count: ', data[0][3])

Avatar | Revenue:  2787965087 | Vote Avg.:  7 | Vote Count:  11800


**5. Найти имена актеров, игравших в Pirates of the Caribbean , сгруппировам по частям фильма и отсортировав по дате релиза**

In [572]:
q = ("""    
    SELECT GROUP_CONCAT(DISTINCT(actor.name), '') AS actors_list, 
       movie.title AS title 
    FROM characters_in_movies, 
         actor, 
         movie 
    WHERE characters_in_movies.char_id=actor.char_id 
      AND characters_in_movies.movie_id=movie.movie_id 
      AND movie.title LIKE '%Pirates of the Caribbean%' 
    GROUP BY title, 
         movie.release_date 
    ORDER BY movie.release_date 
    """)
cursor.execute(q)
data = cursor.fetchall()
for i in data:
    print('Film title: ', i[1])
    print('Actors list: ', i[0])
    print('----------------------------------------------') 

Film title:  Pirates of the Caribbean: The Curse of the Black Pearl
Actors list:  Aaron Douglas,Alan Cumming,Brian Cox,Bryan Singer,Chiara Zanni,Cotter Smith,Daniel Cudmore,Kea Wong,Kelly Hu,Layke Anderson,Michael Reid MacKay,Nolan Gerard Funk,Roger Cross,Shauna Kain,Steve Bacic
----------------------------------------------
Film title:  Pirates of the Caribbean: Dead Man's Chest
Actors list:  Armie Hammer,Barry Pepper,Bob Rumnock,Brad Greenquist,Chad Randall,Gil Birmingham,Helena Bonham Carter,Jack Axelrod,James Badge Dale,Julie Stracener,Laina Loucks,Matt O'Leary,Nick W. Nicholson,Ruth Wilson,Ryan Koenig,Saginaw Grant,Stephen Root,Steve Corona,Tait Fletcher,Tom Wilkinson,W. Earl Brown,William Fichtner
----------------------------------------------
Film title:  Pirates of the Caribbean: At World's End
Actors list:  Chow Yun-fat,David Meunier,Dominic Scott Kay,Ghassan Massoud,Hakeem Kae-Kazim,Ho-Kwan Tse,Jack Davenport,Keith Richards,Peter Donald Badalamenti II,Reggie Lee,Takayo Fische

**6. Допустим, актеры получают с фильма 0.002 от прибыли. Вывести всех актеров и общую прибыль от всех фильмов с оценкой > 7, где они играли. При этом количество символов в имени актера = 10 и дата рождения после 2000 года. Сортировка выдачи по актерам.**

In [573]:
q = ("""
    SELECT n0,
       sum(r1)
    FROM (
        (SELECT name AS n0,
                char_id AS c0
         FROM actor
         WHERE DATE_FORMAT(birth_date, '%Y') > '2000' ) t0
      RIGHT JOIN
        (SELECT characters_in_movies.char_id AS c1,
                movie.revenue * 0.002 AS r1,
                movie.release_date AS d1
         FROM characters_in_movies,
              movie
         WHERE characters_in_movies.movie_id = movie.movie_id
           AND movie.vote_average > 7) t1 ON t1.c1=t0.c0)
    WHERE LENGTH(n0) = 10
    GROUP BY n0
    ORDER BY n0 
    """)
cursor.execute(q)
data = cursor.fetchall()
for i in data:
    print(i[0], 'gained: ', i[1])
    print('---------------------------------------------------------')

Alex Lorre gained:  3191310.500
---------------------------------------------------------
Bill Hader gained:  4490624.076
---------------------------------------------------------
Chris Rock gained:  4931383.464
---------------------------------------------------------
Dan Lauria gained:  1170855.000
---------------------------------------------------------
David Orth gained:  3127871.160
---------------------------------------------------------
Fran Kranz gained:  1260000.000
---------------------------------------------------------
Jim Carter gained:  7667200.000
---------------------------------------------------------
Joey Roche gained:  5557724.400
---------------------------------------------------------
Jon Lovitz gained:  3437057.136
---------------------------------------------------------
Jonah Hill gained:  4490624.076
---------------------------------------------------------
Joseph Rye gained:  3191310.500
---------------------------------------------------------
Josh Pence

**Вывести названия компаний, running revenue по годам для фильмов со средней оценкой  >=7 и <= 8 с разделением по компаниям**

In [658]:
q = ("""
    SELECT name,
       r
    FROM (
        (SELECT comp_id,
                SUM(revenue) OVER (PARTITION BY comp_id
                                   ORDER BY DATE_FORMAT(release_date, "%Y")) AS r
         FROM movie
         WHERE vote_average BETWEEN 7 AND 8 ) t0
      LEFT JOIN
        (SELECT name,
                comp_id AS c1
         FROM companie) t1 ON t0.comp_id=t1.c1)
    """)
cursor.execute(q)
data = cursor.fetchall()
data[0:15] # я не буду совсем все принтить

[('Ingenious Film Partners', Decimal('2787965087')),
 ('Twentieth Century Fox Film Corporation', Decimal('2787965087')),
 ('Dune Entertainment', Decimal('2787965087')),
 ('Lightstorm Entertainment', Decimal('2787965087')),
 ('Walt Disney Pictures', Decimal('961000000')),
 ('Jerry Bruckheimer Films', Decimal('961000000')),
 ('Second Mate Productions', Decimal('961000000')),
 ('Legendary Pictures', Decimal('1084939099')),
 ('Warner Bros.', Decimal('1084939099')),
 ('DC Entertainment', Decimal('1084939099')),
 ('Syncopy', Decimal('1084939099')),
 ('Prime Focus', Decimal('591794936')),
 ('Revolution Sun Studios', Decimal('591794936')),
 ('Heyday Films', Decimal('1405403694')),
 ('DC Comics', Decimal('1405403694'))]

In [659]:
len(data)

5862

А остальное с графиками в приложении   ¯\_(ツ)_/¯